<a href="https://colab.research.google.com/github/duongquangvinh/Fundamental-Machine-Learning-model/blob/main/keras_sequential_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import os

In [2]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()

# scale the data to the range [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# Convert class vectors to binary class matrices.
num_classes = 10
trainY = to_categorical(trainY, num_classes)
testY = to_categorical(testY, num_classes)

# Data normalization
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255
testX /= 255

# Data augmentation
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [3]:
class LeNet(Sequential):
    def __init__(self):
        super(LeNet, self).__init__()
        self.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(32, 32, 3), padding="same"))
        self.add(MaxPool2D(strides=2))
        self.add(Conv2D(filters=48, kernel_size=(5, 5), activation='relu', padding='valid'))
        self.add(MaxPool2D(strides=2))
        self.add(Flatten())
        self.add(Dense(256, activation='relu'))
        self.add(Dense(84, activation='relu'))
        self.add(Dense(10, activation='softmax'))

        self.compile(optimizer=Adam(learning_rate=5e-4),
                    loss=categorical_crossentropy,
                    metrics=['accuracy'])

In [5]:
net = LeNet()
history = net.fit(datagen.flow(trainX, trainY, batch_size=32),
                  steps_per_epoch=len(trainX)/32,
	            validation_data=(testX, testY),
	            epochs=5, verbose=1)

checkpoint_dir = "./checkpoint/"
os.makedirs(checkpoint_dir, exist_ok=True)

net.save_weights(os.path.join(checkpoint_dir, "keras_cifar10.h5"))

# evaluate model and save
_, acc = net.evaluate(testX, testY, verbose=1)
print(f"Accuracy: {acc*100}")

Epoch 1/5
1562/1562 [==============================] - 100s 64ms/step - loss: 2.0741 - accuracy: 0.2292 - val_loss: 1.9024 - val_accuracy: 0.3071
Epoch 2/5
1562/1562 [==============================] - 99s 63ms/step - loss: 1.8453 - accuracy: 0.3279 - val_loss: 1.6810 - val_accuracy: 0.3935
Epoch 3/5
1562/1562 [==============================] - 99s 63ms/step - loss: 1.6962 - accuracy: 0.3851 - val_loss: 1.5806 - val_accuracy: 0.4341
Epoch 4/5
1562/1562 [==============================] - 99s 63ms/step - loss: 1.6157 - accuracy: 0.4187 - val_loss: 1.5139 - val_accuracy: 0.4609
Epoch 5/5
313/313 [==============================] - 5s 16ms/step - loss: 1.4551 - accuracy: 0.4831
Accuracy: 48.30999970436096
